In [2]:
import yfinance as yf
from datetime import datetime, timedelta
from time import sleep
import pandas as pd
from tqdm import tqdm

In [3]:

def get_sp500_tickers():
    url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    tables = pd.read_html(url)
    sp500_table = tables[0]
    return sp500_table['Symbol'].tolist()

In [4]:

def get_ohlcv_yfinance(ticker, period_days=180):
    end = datetime.today()
    start = end - timedelta(days=period_days)

    df = yf.download(
        ticker,
        start=start.strftime('%Y-%m-%d'),
        end=end.strftime('%Y-%m-%d'),
        interval='1d',
        auto_adjust=False,
        progress=False
    )

    return df

In [11]:
def get_multiple_ohlcv_yfinance(tickers, period_days=180, sleep_seconds=0.2):
    end = datetime.today()
    start = end - timedelta(days=period_days)

    all_data = {}
    
    for ticker in tqdm(tickers, desc="Fetching OHLCV"):
        try:
            df = yf.download(
                ticker,
                start=start.strftime('%Y-%m-%d'),
                end=end.strftime('%Y-%m-%d'),
                interval='1d',
                auto_adjust=False,
                progress=False
            )
            if not df.empty:
                all_data[ticker] = df
            else:
                print(f"⚠️ No data for {ticker}")
        except Exception as e:
            print(f"❌ Failed for {ticker}: {e}")
        sleep(sleep_seconds)

    return all_data

In [13]:
tickers = get_sp500_tickers()
data_dict = get_multiple_ohlcv_yfinance(tickers)
print(data_dict['AAPL'].tail())

Fetching OHLCV:  12%|█▏        | 60/503 [00:21<02:50,  2.60it/s]ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['BRK.B']: YFTzMissingError('possibly delisted; no timezone found')


⚠️ No data for BRK.B


Fetching OHLCV:  15%|█▍        | 74/503 [00:27<02:42,  2.64it/s]ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['BF.B']: YFPricesMissingError('possibly delisted; no price data found  (1d 2024-10-09 -> 2025-04-07)')
Fetching OHLCV:  15%|█▍        | 75/503 [00:28<02:36,  2.74it/s]

⚠️ No data for BF.B


Fetching OHLCV: 100%|██████████| 503/503 [03:18<00:00,  2.53it/s]

Price        Adj Close       Close        High         Low        Open  \
Ticker            AAPL        AAPL        AAPL        AAPL        AAPL   
Date                                                                     
2025-03-31  222.130005  222.130005  225.619995  216.229996  217.009995   
2025-04-01  223.190002  223.190002  223.679993  218.899994  219.809998   
2025-04-02  223.889999  223.889999  225.190002  221.020004  221.320007   
2025-04-03  203.190002  203.190002  207.490005  201.250000  205.539993   
2025-04-04  188.380005  188.380005  199.880005  187.339996  193.889999   

Price          Volume  
Ticker           AAPL  
Date                   
2025-03-31   65299300  
2025-04-01   36412700  
2025-04-02   35905900  
2025-04-03  103419000  
2025-04-04  125569000  


In [5]:
def build_price_matrix(data_dict, field="Adj Close"):
    price_df = pd.DataFrame()

    for ticker, df in data_dict.items():
        if field in df.columns:
            s = df[field].copy()
            s.name = ticker
            price_df = pd.concat([price_df, s], axis=1)
    
    # Drop rows with too many missing values (e.g., early dates)
    price_df = price_df.dropna(thresh=int(0.8 * price_df.shape[1]))
    price_df = price_df.fillna(method="ffill").dropna(axis=1)  # ffill and drop any remaining NaNs
    return price_df

In [17]:
from statsmodels.tsa.stattools import coint
import itertools
import pandas as pd
from tqdm import tqdm

def find_cointegrated_pairs(price_df, significance=0.05):
    tickers = price_df.columns
    pairs = []
    scores = []
    pvalues = []

    combos = list(itertools.combinations(tickers, 2))

    for i, (t1, t2) in enumerate(tqdm(combos, desc="Testing cointegration pairs")):
        series1 = price_df[t1]
        series2 = price_df[t2]

        try:
            score, pvalue, _ = coint(series1, series2)
            if pvalue < significance:
                pairs.append((t1, t2))
                scores.append(score)
                pvalues.append(pvalue)
        except Exception:
            continue  # skip bad pairs

    result_df = pd.DataFrame({
        'pair': pairs,
        'coint_score': scores,
        'pvalue': pvalues
    }).sort_values(by='pvalue')

    return result_df

In [4]:
data = build_price_matrix(data_dict)
pairs = find_cointegrated_pairs(price_df=data)

NameError: name 'build_price_matrix' is not defined

In [8]:
data = pd.read_csv('sp500_historical_data.csv')
pairs = pd.read_csv('sp500_pairs.csv')

In [ ]:
import matplotlib.pyplot as plt
import statsmodels.api as sm
import numpy as np

def plot_cointegrated_pair(price_df, ticker1, ticker2):
    series1 = price_df[ticker1]
    series2 = price_df[ticker2]

    # Step 1: Normalize prices for comparison
    norm1 = series1 / series1.iloc[0]
    norm2 = series2 / series2.iloc[0]

    # Step 2: OLS to estimate hedge ratio
    X = sm.add_constant(series2)
    model = sm.OLS(series1, X).fit()
    hedge_ratio = model.params[1]
    spread = series1 - hedge_ratio * series2
    zscore = (spread - spread.mean()) / spread.std()

    # Step 3: Plot normalized price series
    fig, axs = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

    axs[0].plot(norm1, label=ticker1)
    axs[0].plot(norm2, label=ticker2)
    axs[0].set_title(f"Normalized Prices: {ticker1} vs {ticker2}")
    axs[0].legend()

    # Step 4: Plot z-score of spread
    axs[1].plot(zscore, label='Z-Score', color='purple')
    axs[1].axhline(0, color='black', linestyle='--')
    axs[1].axhline(1, color='green', linestyle='--')
    axs[1].axhline(-1, color='red', linestyle='--')
    axs[1].set_title("Z-Score of Spread")
    axs[1].legend()

    plt.tight_layout()
    plt.show()

top_pair = pairs.iloc[0]['pair']
for i in range(len(pairs)):
    print(pairs.iloc[i])
plot_cointegrated_pair(data, top_pair[0], top_pair[1])

In [27]:
data.to_csv('sp500_historical_data.csv')
pairs.to_csv('sp500_pairs.csv')

In [19]:
#print(len(data_dict.items()))
print(len(data.columns))

502


In [35]:
from statsmodels.tsa.stattools import adfuller

def test_cointegrated_pair(series1, series2, adf_thresh=0.01, max_zstd=1.25, min_cross=10):
    X = sm.add_constant(series2)
    model = sm.OLS(series1, X).fit()
    hedge_ratio = model.params.iloc[1]
    spread = series1 - hedge_ratio * series2
    z = (spread - spread.mean()) / spread.std()

    # ADF test on spread
    adf_stat, adf_pval, *_ = adfuller(spread)
    if adf_pval > adf_thresh:
        return False

    # Volatility filter
    if z.std() > max_zstd or z.abs().max() > 4:
        return False

    # Zero crossings
    crossings = ((z.shift(1) * z) < 0).sum()
    if crossings < min_cross:
        return False

    return True


def calculate_half_life(spread):
    spread_lag = spread.shift(1)
    spread_lag.iloc[0] = spread_lag.iloc[1]
    delta = spread - spread_lag

    model = sm.OLS(delta, sm.add_constant(spread_lag)).fit()
    lambda_ = model.params.iloc[1]
    if lambda_ >= 0:
        return np.inf
    half_life = -np.log(2) / lambda_
    return half_life



def stable_cointegration(series1, series2, adf_thresh=0.01):
    n = len(series1)
    half = n // 2

    s1a, s2a = series1[:half], series2[:half]
    s1b, s2b = series1[half:], series2[half:]

    def adf_test(s1, s2):
        model = sm.OLS(s1, sm.add_constant(s2)).fit()
        spread = s1 - model.params.iloc[1] * s2
        return adfuller(spread)[1]  # p-value

    return (adf_test(s1a, s2a) < adf_thresh) and (adf_test(s1b, s2b) < adf_thresh)




filtered_pairs = []
import ast
print(len(pairs))
for _, row in pairs.iterrows():
    p = ast.literal_eval(row['pair'])
    t1, t2 = p
    series1 = data[t1]
    series2 = data[t2]

    # Step 1: Basic filters (ADF, z-score stats, crossings)
    if not test_cointegrated_pair(series1, series2):
        continue

    # Step 2: Check for stable cointegration across time
    if not stable_cointegration(series1, series2):
        continue

    # Step 3: Calculate spread and half-life
    X = sm.add_constant(series2)
    model = sm.OLS(series1, X).fit()
    hedge_ratio = model.params.iloc[1]

    spread = series1 - hedge_ratio * series2
    half_life = calculate_half_life(spread)

    # Step 4: Filter based on half-life (e.g., must revert in 2–20 days)
    if half_life < 2 or half_life > 20:
        continue

    # Passed all filters
    filtered_pairs.append((t1, t2))



5655


In [36]:
print(filtered_pairs)
print(len(filtered_pairs))

[('APO', 'CRM'), ('APH', 'CDNS'), ('AMD', 'BXP'), ('APH', 'PNR'), ('DPZ', 'VRSK'), ('NRG', 'WFC'), ('CRM', 'SYF'), ('AOS', 'AMD'), ('ENPH', 'WTW'), ('NDAQ', 'NTRS'), ('KEY', 'MTB'), ('K', 'VRSN'), ('ERIE', 'KHC'), ('D', 'NUE'), ('CNP', 'TGT'), ('KO', 'WST'), ('ADM', 'BALL'), ('ENPH', 'VRSK'), ('PAYX', 'SWKS'), ('AON', 'DGX'), ('DHR', 'KR'), ('HD', 'NCLH'), ('ETR', 'FOX'), ('PPL', 'RSG'), ('JKHY', 'VST'), ('AMD', 'IRM'), ('CVX', 'REG'), ('AIG', 'ANET'), ('ABNB', 'SPG'), ('BBY', 'CZR'), ('NSC', 'ORCL'), ('AMP', 'PNC'), ('PODD', 'JPM'), ('ALLE', 'ADM'), ('FSLR', 'K'), ('PH', 'TFC'), ('CPAY', 'TFC'), ('CSCO', 'FTNT'), ('CCL', 'CRM'), ('ETN', 'FDX'), ('FE', 'PHM'), ('MU', 'SWK'), ('BEN', 'TYL'), ('BBY', 'WDC'), ('ALLE', 'BXP'), ('D', 'PFG'), ('D', 'XOM'), ('FSLR', 'MLM'), ('D', 'ES'), ('D', 'LIN'), ('GLW', 'KEYS'), ('EW', 'GD'), ('CVS', 'DUK'), ('EXPD', 'SBAC'), ('DLR', 'TROW'), ('CVX', 'FITB'), ('FSLR', 'IRM'), ('DLR', 'PNR'), ('CVX', 'SYF'), ('CVX', 'CRM'), ('CVX', 'STT'), ('ETN', 'IEX'),

In [37]:
df = pd.DataFrame(filtered_pairs, columns=["asset_1", "asset_2"])
df["type_1"] = "equity"
df["type_2"] = "equity"

df.to_csv('pairs.csv',index=False)